In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, concatenate, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import Sequence, plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine
from sklearn.metrics import confusion_matrix

Podczas zajęć będziemy używać TensorFlow oraz Kerasa, ale projekt może być zrobiony w dowolnej technologii. Można nawet użyć czystego numpy, albo C czy Assemblera

TensorFlow z automatu rezerwuje sobie sporo pamięci na GPU, aby temu zapobiec należy ustawić parametr memory_growth, który pozwala mu dynamicznie zwiększać zajętość pamięci w zależności od potrzeb

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, enable=True)

Zanim przejdziemy do sieci neuronowych zacznijmy od pojedynczego perceptronu. To prosty model matematyczny, który liczy sumę ważoną a następnie przykłada wybraną funkcję zwaną funkcją aktywacji

In [ ]:
model = Sequential()
model.add(Dense(1, input_shape=(2,))) 
# that's how we create a single layer. First parameter specifies how many neurons do we want, since we want to have
# a single perceptron we use 1. Then we specify the input shape so the number of variables 
model.summary()

Powyżej stworzyliśmy prostą sieć neuronową z jednym neuronem, która przyjmuje wektor o długości dwa na wejściu. Jak widać taka sieć ma 3 parametry. Dwa z nich to wagi przypisane do każdego z wejść. Dodatkowy parametr, tak zwany bias, to stała

Wzór na pojedynczy neuron można zdefiniować tak
$$ \sum (x_i*w_i) + bias$$

Sprawdźmy czy to rzeczywiśie prawda

In [ ]:
model.weights

To są wagi naszego modelu - bias oraz wagi wejść. Są one generowane w sposób losowy, więc przy każdm wywołaniu mogą być zupełnie inne

In [ ]:
x = np.array([[2,1]]) # dummy input for calculations
print(model(x))
print(x @ model.weights[0].numpy() + model.weights[1].numpy())

Wygląda na to, że rzeczywiście dokładnie takie obliczenia mają miejsce, gdyż wynik wywołania modelu jest tożsamy z przeprowadzonymi obliczeniami. Znak @ odpowiada za mnożenie macierzy, jeśli nie ufasz możesz sprawdzić implementując swoje mnożenie macierzy

Ale miała być jeszcze funkcja aktywacji

Zgadza się, tutaj używamy liniowej fukncji f(x) = x, stąd nie ma ona wpływu na wynik, ale można użyć praktycznie dowolnej funkcji, która jest później aplikowana do wyniku sumy ważonej, zaktualizujmy więc nasz wzór na perceptron
$$ f(\sum (x_i*w_i) + bias)$$

In [ ]:
model = Sequential()
model.add(Dense(1, 'tanh', input_shape=(2,))) # the second parameter specifies the activation function
model.summary()

In [ ]:
model.weights

In [ ]:
model.predict(x), x @ model.weights[0].numpy() + model.weights[1].numpy()

Teraz wyniki się rozjechały. To dlatego, że model przykłada jeszcze funkcję tanh. Musimy ją dodać do naszych obliczeń

In [ ]:
np.tanh(x @ model.weights[0].numpy() + model.weights[1].numpy())

Wyniki mogą się odrobinę różnić, wynika to z dokładności numerycznej i różnej implementacji funkcji tanh.


Jasne już jest jak działa neuron z ustalonymi wagami, ale skąd je wziąć? 

Pomysł jest trywialnie prosty. Potrzebujemy zbioru referencyjnego (treningowego) w którym mamy predyktory ($x$) oraz target ($y$). Następnie szukamy takich wag, które jak najwierniej mapują $x$ na $y$. W naszym przykładzie spróbujemy przewidzieć cenę mieszkania ($y$) na podstawie jego powierzchni ($x$). Oczywiście w tego typu zadaniu warto użyć więcej cech jak piętro, lokalizacja, rok budowy itp, ale dla prostoty użyjemy tylko jednej cechy.


In [ ]:
np.random.seed(41)
x = np.random.rand(200,1)*20+50

In [ ]:
np.random.seed(27)
y = .5*x + np.random.rand(*x.shape)*3 + np.log(x-49)*2

In [ ]:
plt.plot(x,y, '.')
plt.xlabel("$m^2$")
plt.ylabel("price in bitcoins")
plt.show()

Stworzyliśmy zbiór danych, teraz potrzebujemy sieci neuronowej. Jaka powinna być funkcja aktywacji?

In [ ]:
model = Sequential()
model.add(Dense(1, input_shape=(1,))) #now we have only one input parameter
model.summary()

In [ ]:
model.predict(x)

In [ ]:
plt.plot(x,y, '.', label='data')
plt.plot(np.sort(x,0), model.predict(np.sort(x,0)), label='prediction')
plt.legend()
plt.grid()
plt.show()

To nasza predykcja. Prawdopodobnie nie jesteśmy nawet blisko. Nie ma się co dziwić, w końcu mamy losowe wagi. Teraz musimy zdefiniować funkcję określającą na ile nasza predykcja odbiega od tej oczekiwanej - funkcja straty. Kiedy będziemy ją mieli możliwe będzie ocenienie danego wektora wag, więc problem znalezienia odpowiednich parametrów staje się zwykłym zadaniem optymalizacyjnym. Należy tak dobrać parametry, żeby zminimalizować funkcję straty. W praktyce korzysta się z faktu iż funkcja straty jest ciągła i różniczkowalna dzięki czemu można użyć algorytm spadku gradientu, ale nic nie stoi na przeszkodzie aby użyć algorytm genetyczny, czy nawet random search

Pierwszym pomysłem na funkcję straty może być $prediction - y$. Oczywiście to podejście zawiedzie, gdyż pozytywne i negatywne błędy będą się znosić nawzajem. Jednym z rozwiązać może być policzenie modułu z tej miary. Jest to już poprawne podejście, jednak funkcja ta nie jest różniczkowalna, stąd w praktyce często stosuje się kwadrat tej miary $(prediction - y)^2$

In [ ]:
mse = lambda x,y: ((y - x)**2).mean() # our loss function

Sprawdźmy czy rzeczywiście można użyć random searcha do treningu sieci neuronowej

In [ ]:
bestWeights = model.get_weights()
pred = model.predict(x, verbose=0)
bestError = mse(y, pred)
bestError

for _ in range(200):
    weights = [np.random.randn(1,1), np.random.randn(1)]
    model.set_weights(weights)
    pred = model.predict(x, verbose=0)
    err = mse(y, pred)
    if err < bestError:
        bestError = err
        bestWeights = model.get_weights()
        print(err)


In [ ]:
model.set_weights(bestWeights)
plt.plot(x,y, '.', label='data')
plt.plot(np.sort(x,0), model.predict(np.sort(x,0)), label='prediction')
plt.legend()
plt.grid()
plt.show()

Wygląda nieźle, no ale oczywiście nie jest to najlepsze podejście. Nie dość, że istnieją lepsze techniki, to są już zaimplementowane, więc nie trzeba się samemu wysilać

Przy treninu specyfikuje się dwa podstawowe parametry rozmiar batcha oraz liczba epok. Liczba epok mówi nam o tym ile razy przeiterujemy się po całym zbiorze danych w fazie treningu, rozmiar batcha określa ile wierszy na raz będziemy rozważać podczas jednego kroku obliczania aktualizacji wag. Jak łatwo się domyślić liczba aktualizacji to liczba epok * rozmiar danych / batch size

Zanim użyjemy gotowej funkcji treningu musimy skompilować model i zdefiniować funkcję straty

In [ ]:
model = Sequential()
model.add(Dense(1, input_shape=(1,)))
model.compile(loss='mse', metrics='mae')
model.fit(x,y, epochs=300, batch_size=16)

Jak dobrać te parametry? W uproszczeniu batch size najczęściej chcemy mieć tak duży jak to możliwe - ogranicza nas tutaj pamięć, zazwyczaj karty graficznej. Liczba epok to bardziej złożony temat. Jak widać na naszym przykładzie w pewnym momencie dalszy trening nie ma już sensu, tylko jak to wykryć? Są gotowe zaimplementowace funkcje - callbacki. Jedna z nich EarlyStopping przerywa trening, jeśli funkcja straty nie poprawiła się przez założoną liczbę epok. Warto też rozważyć użycie ReduceLROnPlateau, który w analogicznej sytuacji zmniejsza stałą uczenia pozwalając na drobniejsze kroki - precyzyjniesze zbliżenie się do punktu optimum

In [ ]:
early = EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)
reduce = ReduceLROnPlateau(monitor='loss', patience=6)

model = Sequential()
model.add(Dense(1, input_shape=(1,)))
model.compile(loss='mse', metrics='mae')
model.fit(x,y, epochs=1000, batch_size=16, callbacks=[early, reduce])

In [ ]:
plt.plot(x,y, '.', label='data')
plt.plot(np.sort(x,0), model.predict(np.sort(x,0)), label='prediction')
plt.legend()
plt.grid()
plt.show()

Wynik nie jest idealny, ale w tym wypadku nie da się lepiej. Po prostu model liniowy nie jest w stanie wierniej zmapować $x$ na $y$. Nie istnieje prosta, która dużo lepiej dopasuje się do danych. Możemy np zwiększyć liczbę neuronów. Zazwyczaj zorganizowane są one w warstwy, wszystkie neurony z warstwy poprzedniej są połączone z wszystkim neuronami z warstwy następnej

Możemy też zmienić funkcję aktywacji. Najczęściej używane to:

In [ ]:
print("linear")
r = np.linspace(-7,7)
plt.plot(r,r)
plt.grid()
plt.show()
print("tanh")
plt.plot(r, np.tanh(r))
plt.grid()
plt.show()
print("sigmoid")
plt.plot(r, 1/(1 + np.exp(-r)))
plt.grid()
plt.show()
print("relu")
plt.plot(r, np.where(r<0,0,r))
plt.grid()
plt.show()
print("leakyrelu")
plt.plot(r, np.where(r<0,r*.1,r))
plt.grid()
plt.show()

Jak zdecdować którą wybrać? Jak widać niektóre z nich mają ograniczony zakres wartości np sigmoid od 0 do 1, dlatego nadaje się on idealnie do sytuacji, w których wynik ma być traktowany jak prawdopodobieństwo. 

Przy kilku warstwach nie opłaca się stosować liniowej aktywacji - wiesz dlaczego?

Dobry pierwszy wybór to zazwyczaj relu

Proces doboru wag używa pochodnych funkcji aktywacji. Jak widać dla większości z nich największa zmienność pochodnej jest w okolicy zera dlatego chcemy mieć wartości w tej okolicy. W tym celu dobrze jest odpowiednio przeskalować dane

In [ ]:
ss_x = StandardScaler()
ss_y = StandardScaler()

transformed_x = ss_x.fit_transform(x)
transformed_y = ss_y.fit_transform(y)

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(1,))) # 64 neurons in the first layer
model.add(Dense(1)) # no need to specify input shape. Why?

model.compile(loss='mse', metrics='mae')

early = EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)
reduce = ReduceLROnPlateau(monitor='loss', patience=6)

model.fit(transformed_x,transformed_y, epochs=500, batch_size=16, callbacks=[early, reduce])

In [ ]:
plt.plot(transformed_x,transformed_y, '.', label='data')
plt.plot(np.sort(transformed_x, 0), model.predict(np.sort(transformed_x, 0)), label='prediction')
plt.legend()
plt.grid()
plt.show()

Funkcja sortująca jest tylko po to, żeby narysować ładną linię. Jeśli jej nie użyjemy to można narysować punkty, ale połączenie ich odcinkami da niezbyt dobry rezultat

In [ ]:
plt.plot(transformed_x,transformed_y, '.', label='data')
plt.plot(transformed_x, model.predict(transformed_x), label='prediction')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.plot(transformed_x,transformed_y, '.', label='data')
plt.plot(transformed_x, model.predict(transformed_x), '.', label='prediction')
plt.legend()
plt.grid()
plt.show()

# Task 1
Potestuj różne architektury https://playground.tensorflow.org/

# Task 2
Zobacz jaka architektura działa najlpiej dla naszego problemu

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(1,))) 
model.add(Dense(32, activation='relu')) 
model.add(Dense(1))

model.compile(loss='mse', metrics='mae')

early = EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)
reduce = ReduceLROnPlateau(monitor='loss', patience=6)

model.fit(transformed_x,transformed_y, epochs=500, batch_size=16, callbacks=[early, reduce])

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(1,))) 
model.add(Dense(128, activation='relu')) 
model.add(Dense(1))

model.compile(loss='mse', metrics='mae')

early = EarlyStopping(monitor='loss', patience=15, restore_best_weights=True)
reduce = ReduceLROnPlateau(monitor='loss', patience=6)

model.fit(transformed_x,transformed_y, epochs=500, batch_size=16, callbacks=[early, reduce])

Wybranie liczby neuronów i warstw jest samo w sobie zagadnieniem optymalizacyjnym. Nie ma ścisłych reguł, generalnie należy zwiększać rozmiar siecii jeśli poprawia się jakość, ale jednocześnie uważać na przeuczenie. Zawsze używaj osobnego zbioru walidacyjnego i testowego.

Pamiętaj, że nawet najbardziej złożona sieć nadal działa jak pojedynczy perceptron -- to po prostu równanie matematyczne z wagami wybranymi przez algorytm optymalizacji.

Tip: skoro dobór architektóry to problem optymalizacyjny to możemy użyć algorytmu optymalizującego żeby go rozwiązać. Oczywistym pomysłem na reprezentację jest wektor z liczbą neuronów w poszczególnych warstwach. Niestety to podejście ma swoje minusy. W skrajnej sytuacji np w drugiej warstwie może być 0 neuronów a w trzeciej >0. W ogólności w tego typu sieciach należy raczej unikać zwiększania liczby neuronów w kolejnej warstwie. Ciekawym podejściem jest taka reprezentacja w której pierwsza liczba to liczba neuronów w pierwszej warstwie a kolejne liczby to procent neuronów z warstwy poprzedniej. Przykładowo $[64, 1, .5, .5, 0, 0]$ oznacza model z 4 warstwami w których jest odpowiednio 64, 64, 32, oraz 16 neuronów.

In [ ]:
np.int = np.int64
def loss(x):
    model = Sequential()
    neurons = int(x[0])
    model.add(Dense(neurons, activation='relu', input_shape=(train_x.shape[1],)))
    for i in x[1:]:
        neurons = int(i*neurons)
        if neurons < 2:
            break
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', metrics='mae')

    early = EarlyStopping(patience=15, restore_best_weights=True)
    reduce = ReduceLROnPlateau(patience=6)

    model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=500, batch_size=16, callbacks=[early, reduce], verbose=0)
    res = model.evaluate(val_x, val_y, verbose=0)
    print(res)
    return res[0]

In [ ]:
from skopt import gp_minimize

np.random.seed(31)
idx = np.arange(len(x))
np.random.shuffle(idx)

train = idx[:int(.8*len(x))]
val = idx[int(.8*len(x)):int(.9*len(x))]
test = idx[int(.9*len(x)):]

train_x, train_y = transformed_x[train], transformed_y[train]
val_x, val_y = transformed_x[val], transformed_y[val]
test_x, test_y = transformed_x[test], transformed_y[test]

result = gp_minimize(loss, [(16,256), (0,1.0), (0,1.0), (0,1.0)], n_calls=30)

In [ ]:
result.x

Aby uniknąć przeuczenia dzielimy zbiór danych na treningowy, walidacyjny i testowy. W większości przypadków warto wcześniej przemieszać dane aby uniknąć problemów związanych z nieprzypadkowym ułożeniem danych, przykładowo mogą być posortowane według targetu.

In [ ]:
np.random.seed(31)
idx = np.arange(len(x))
np.random.shuffle(idx)
idx

In [ ]:
train = idx[:int(.8*len(x))]
val = idx[int(.8*len(x)):int(.9*len(x))]
test = idx[int(.9*len(x)):]

In [ ]:
train_x, train_y = transformed_x[train], transformed_y[train]
val_x, val_y = transformed_x[val], transformed_y[val]
test_x, test_y = transformed_x[test], transformed_y[test]

Problem jest tu taki, że skorzystaliśmy ze wszystkich danych, żeby wytrenować skaler danych, nie jest to duży błąd, ale należy unikać tego typu rozwiązań. 

#### Task
Podziel oryginalne dane na poszczególne zbiory i odpowiednio je przeskaluj

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(1,))) 
model.add(Dense(1, activation='linear'))

model.compile(loss='mse', metrics='mae')

early = EarlyStopping(patience=15, restore_best_weights=True) # we don't specify monitor, by default it's val_loss
reduce = ReduceLROnPlateau(patience=6)

model.fit(train_x,train_y, validation_data=(val_x, val_y), epochs=500, batch_size=16, callbacks=[early, reduce])

In [ ]:

plt.plot(test_x,test_y, '.', label='data')
plt.plot(np.sort(test_x, 0), model.predict(np.sort(test_x, 0)), label='prediction')
plt.legend()
plt.grid()
plt.show()

Dwa popularne modele na tworzenie sieci neuronowych to Sequential i Model. Pierwszy dobrze działa dla prostych sekwencyjnych sieci, za pomocą drugiego można tworzyć bardzie złożone modele.

W sekwencyjnych modelach zakładamy, że warstwy są wywoływana po kolei jedna po drugiej. W Model, każdą warstwę traktujemy jaku funkcję i można je dowolnie zagnieżdżać

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(1,))) 
model.add(Dense(32, activation='relu', input_shape=(1,))) 
model.add(Dense(16, activation='relu', input_shape=(1,))) 
model.add(Dense(1, activation='linear'))

In [ ]:
plot_model(model)

In [ ]:
inputLayer = Input(shape=(1,))
dense1 = Dense(64, activation='relu')(inputLayer)
dense2 = Dense(32, activation='relu')(dense1)
dense3 = Dense(16, activation='relu')(dense2)
dense4 = Dense(1, activation='relu')(dense3)
model = Model(inputs=inputLayer, outputs=dense4)

In [ ]:
plot_model(model)

In [ ]:
inputLayer = Input(shape=(1,))
dense1 = Dense(64, activation='relu')(inputLayer)
dense2 = Dense(32, activation='relu')(dense1)
concat = Concatenate()([dense1, dense2])
dense3 = Dense(16, activation='relu')(concat)
dense4 = Dense(1, activation='relu')(dense3)
model = Model(inputs=inputLayer, outputs=dense4)

In [ ]:
plot_model(model)

# Task 3
Stwórz model z dwoma wejściami, pierwsze przetwarzane przez 3 warstwy Dense, drugie przez dwie. Następnie połącz je, przepuść przez 2 warswy Dense i rozdziel na dwa wyjścia

Generator danych to przydatne narzędzie do treningu sieci neuronowych. Pozwala ono na tworzenie każdego batcha danych niezależnie. Dzięki temu nie ma konieczności trzymania całego zbioru w pamięci co może być problematyczne np przy dużym zbiorze z obrazkami. Można skorzystać z tej funkcjonalności także przy augmentacji danych lub ich generowaniu w locie na podstawie zdefiniowanej funkcji.

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, x, y, batch_size, shuffle=True):
        self.x = x
        self.y = y
        self.indexes = np.arange(len(y))
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.indexes) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch. 
        # During training and prediction this function will be called in range(0, __len__())
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, idx):
        X = np.empty((self.batch_size, 1))
        y = np.empty((self.batch_size), )

        for i, ID in enumerate(idx):
            # Store sample
            X[i,] = self.x[ID]

            # Store class
            y[i] = self.y[ID]

        return X, y

In [ ]:
trainGenerator = DataGenerator(train_x, train_y, 16)
valGenerator = DataGenerator(val_x, val_y, 1, shuffle=False)

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(1,))) 
model.add(Dense(1, activation='linear'))

model.compile(loss='mse', metrics='mae')

early = EarlyStopping(patience=15, restore_best_weights=True) # we don't specify monitor, by default it's val_loss
reduce = ReduceLROnPlateau(patience=6)

model.fit(trainGenerator, validation_data=valGenerator, epochs=500, batch_size=16, callbacks=[early, reduce])

In [ ]:
plt.plot(test_x,test_y, '.', label='data')
plt.plot(np.sort(test_x, 0), model.predict(np.sort(test_x, 0)), label='prediction')
plt.legend()
plt.grid()
plt.show()

Oczywiście w tym wypadku generator nie ma zbyt wiele sensu. Wszystkie dane i tak są w pamięci i nic z nimi nie robimy. Natomiast jest to baza, którą można wykorzystać gdy zajdzie taka potrzeba

# Task 4
Używając sieci neuronowej zapredykuj jakość wina:
 - traktując target jako zmienną ciągłą (regresja)
 - traktując target jako zmienną dyskretną (klasyfikacja)

In [ ]:
data = load_wine()

In [ ]:
x = data['data']
y = data['target']

In [ ]:
x

In [ ]:
y

In [ ]:
#try also this model
#check why and how it works
model = Sequential()
model.add(Dense(16, 'relu', input_shape=(13,)))
model.add(Dense(3, 'softmax'))
model.compile(loss='sparse_categorical_crossentropy', metrics=['acc'])
history = model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=500, batch_size=16, callbacks=[early, reduce])

## Projekt
Podczas pierwszych zajęć zadaniem było stworzenie w numpy funkcji, która wyliczy prawdopodobieństwo wygrania wyborów. Stwórz i wytrenuj sieć neuronową, która realizuje to zadanie. Oczywiście sieć będzie zwracać przybliżone wartości, za to znacznie szybciej. Załóż, że liczba potencjalnych kandydatów będzie nie większa niż 20. Jeśli model na sztywno przyjmuje na wejściu dane dla 20 potencjalnych kandydatów i jeśli będzie ich mniej to odpowiednie elemnty wejścia są wyzerowane to zadbaj o to, żeby na wyjściu dla nich również były zera

Dodatkowe punkt za następujące elementy:
 - Wykorzystanie generatora danych
 - Dodanie warstwy lambda, która odpowiada za wstawienie zer w odpowiednie miejsca jeśli liczba kandydatów jest mniejsza niż 20